In [ ]:
# @title Check RAM and Disk
!cat /proc/meminfo | head -n 5
!df -h /content

MemTotal:       13286964 kB
MemFree:          260216 kB
MemAvailable:   11994108 kB
Buffers:          189368 kB
Cached:         11316936 kB
Filesystem      Size  Used Avail Use% Mounted on
overlay         108G   69G   40G  64% /


In [ ]:
# @title Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Install Kraken2 with micromamba
%%bash
set -euo pipefail
curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest | tar -xvj -C /usr/local/bin bin/micromamba --strip-components=1
micromamba create -y -n k2 -c conda-forge -c bioconda kraken2
micromamba run -n k2 kraken2 --version

bin/micromamba
conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache
bioconda/linux-64                                           Using cache
bioconda/noarch                                             Using cache


Transaction

  Prefix: /root/.local/share/mamba/envs/k2

  Updating specs:

   - kraken2


  Package                         Version  Build               Channel           Size
───────────────────────────────────────────────────────────────────────────────────────
  Install:
───────────────────────────────────────────────────────────────────────────────────────

  + _libgcc_mutex                     0.1  conda_forge         conda-forge     Cached
  + _openmp_mutex                     4.5  2_gnu               conda-forge     Cached
  + blast                          2.17.0  h66d330f_0          bioconda        Cached
  + bzip2                           1.0.8  hda65f42_8          conda-for

In [ ]:
# @title File paths
R1_in_drive = "/content/drive/MyDrive/ECES650/evol1.sorted.unmapped.R1.fastq"  #@param {type:"string"}
R2_in_drive = "/content/drive/MyDrive/ECES650/evol1.sorted.unmapped.R2.fastq"  #@param {type:"string"}
OUT_DIR_DRIVE = "/content/drive/MyDrive/ECES650/kraken_out"  #@param {type:"string"}

import os, shutil
os.makedirs(OUT_DIR_DRIVE, exist_ok=True)

# Copy FASTQs to /content for faster local I/O
R1 = "/content/" + os.path.basename(R1_in_drive)
R2 = "/content/" + os.path.basename(R2_in_drive)
shutil.copy2(R1_in_drive, R1)
shutil.copy2(R2_in_drive, R2)
print("Copied to:", R1)
print("Copied to:", R2)

Copied to: /content/evol1.sorted.unmapped.R1.fastq
Copied to: /content/evol1.sorted.unmapped.R2.fastq


In [ ]:
# @title Download DB
DB_URL = "https://genome-idx.s3.amazonaws.com/kraken/k2_standard_16_GB_20251015.tar.gz"  #@param {type:"string"}
DB_DIR = "/content/k2db"

import os, subprocess, shlex
os.makedirs(DB_DIR, exist_ok=True)

if not DB_URL:
    print("Please set DB_URL from https://benlangmead.github.io/aws-indexes/k2/ (e.g., Standard-16 tar.gz).")
else:
    tar_name = os.path.join(DB_DIR, os.path.basename(DB_URL) or "k2_standard_16.tar.gz")
    print("Downloading:", DB_URL)
    subprocess.run(shlex.split(f"wget -O {tar_name} '{DB_URL}'"), check=True)
    print("Unpacking…")
    subprocess.run(shlex.split(f"tar -xvzf {tar_name} -C {DB_DIR}"), check=True)
    print("Contents:")
    subprocess.run(["bash", "-lc", f"ls -lh {DB_DIR}"])

Downloading: https://genome-idx.s3.amazonaws.com/kraken/k2_standard_16_GB_20251015.tar.gz
Unpacking…
Contents:


In [ ]:
# @title Execution
%%bash -s "$R1" "$R2"
set -euo pipefail

BASE="/content/k2db"

# Prefer a flat DB in BASE (hash.k2d/opts.k2d/taxo.k2d live directly under BASE)
if [[ -f "$BASE/hash.k2d" && -f "$BASE/opts.k2d" && -f "$BASE/taxo.k2d" ]]; then
  DBDIR="$BASE"
else
  # Otherwise, find the folder that contains hash.k2d
  DBHASH_PATH="$(find "$BASE" -type f -name 'hash.k2d' -print -quit || true)"
  if [[ -z "$DBHASH_PATH" ]]; then
    echo "ERROR: Could not locate Kraken2 DB (hash.k2d) under $BASE."
    echo "Make sure the Standard-16 tar.gz was successfully extracted into $BASE."
    exit 2
  fi
  DBDIR="$(dirname "$DBHASH_PATH")"
fi

OUT="/content/kraken_out"
mkdir -p "$OUT"

echo "Using DBDIR: $DBDIR"
ls -lh "$DBDIR" | sed -n '1,20p'

# Positional args from Python line (-s "$R1" "$R2")
R1_FILE="$1"
R2_FILE="$2"

# Auto-detect compression by filename
GZ_FLAG=""
[[ "$R1_FILE" == *.gz || "$R2_FILE" == *.gz ]] && GZ_FLAG="--gzip-compressed"
echo "Inputs:"
echo "  R1: $R1_FILE"
echo "  R2: $R2_FILE"
echo "Compression flag: ${GZ_FLAG:-<none>}"

# Run Kraken2 (no --fastq-input; enable memory mapping to avoid OOM on Colab Pro)
micromamba run -n k2 kraken2 \
  --db "$DBDIR" \
  --paired $GZ_FLAG \
  --memory-mapping \
  --report "$OUT/evol1.kraken.report" \
  --output "$OUT/evol1.kraken.output" \
  --use-names \
  "$R1_FILE" "$R2_FILE"

echo "Top of report:"
head -n 20 "$OUT/evol1.kraken.report" || true

Using DBDIR: /content/k2db
total 27G
-rw-rw-rw- 1 3948 users 5.6M Oct 15 15:19 database100mers.kmer_distrib
-rw-rw-rw- 1 3948 users 5.1M Oct 15 15:36 database150mers.kmer_distrib
-rw-rw-rw- 1 3948 users 4.6M Oct 15 15:53 database200mers.kmer_distrib
-rw-rw-rw- 1 3948 users 4.2M Oct 15 16:11 database250mers.kmer_distrib
-rw-rw-rw- 1 3948 users 3.9M Oct 15 16:29 database300mers.kmer_distrib
-rw-rw-rw- 1 3948 users 5.9M Oct 15 14:47 database50mers.kmer_distrib
-rw-rw-rw- 1 3948 users 5.8M Oct 15 15:03 database75mers.kmer_distrib
-rw-rw-rw- 1 3948 users  15G Oct 15 13:55 hash.k2d
-rw-rw-rw- 1 3948 users 3.8M Oct 15 13:57 inspect.txt
-rw-r--r-- 1 root root     0 Nov  9 03:22 k2_standard_16_20251015.tar.gz
-rw-r--r-- 1 root root   12G Oct 29 21:45 k2_standard_16_GB_20251015.tar.gz
-rw-rw-rw- 1 3948 users 2.9M Oct 15 13:57 ktaxonomy.tsv
-rw-rw-rw- 1 3948 users  53M Oct 15 16:29 library_report.tsv
-rw-r--r-- 1 3948 users 260M Oct 15 16:29 names.dmp
-rw-r--r-- 1 3948 users 196M Oct 15 16:29 nod

Loading database information... done.
17692 sequences (0.85 Mbp) processed in 6.412s (165.6 Kseq/m, 7.93 Mbp/m).
  738 sequences classified (4.17%)
  16954 sequences unclassified (95.83%)


In [ ]:
# @title Preview top taxa
import pandas as pd, os
report_path = "/content/kraken_out/evol1.kraken.report"
cols = ["percent","reads_clade","reads_taxon","rank","ncbi_taxid","name"]
df = pd.read_csv(report_path, sep="\t", header=None, names=cols, engine="python")
df["name"] = df["name"].str.strip()
df_top = df.sort_values("percent", ascending=False).head(25)
df_top

,percent,reads_clade,reads_taxon,rank,ncbi_taxid,name
0,95.83,16954,16954,U,0,unclassified
1,4.17,738,0,R,1,root
2,4.15,735,0,R1,131567,cellular organisms
3,3.88,687,0,R2,2,Bacteria
4,3.79,670,0,K,3379134,Pseudomonadati
5,3.77,667,0,P,1224,Pseudomonadota
6,3.75,664,0,C,1236,Gammaproteobacteria
7,3.73,660,0,O,91347,Enterobacterales
8,3.72,659,616,F,543,Enterobacteriaceae
68,0.27,48,0,R3,33154,Opisthokonta


In [ ]:
# @title Save outputs
import shutil, os
OUT_LOCAL = "/content/kraken_out"
OUT_DIR_DRIVE = OUT_DIR_DRIVE  # from earlier form cell
os.makedirs(OUT_DIR_DRIVE, exist_ok=True)

for fname in ["evol1.kraken.report", "evol1.kraken.output"]:
    src = os.path.join(OUT_LOCAL, fname)
    dst = os.path.join(OUT_DIR_DRIVE, fname)
    if os.path.exists(src):
        shutil.copy2(src, dst)
        print("Saved:", dst)
    else:
        print("Missing:", src)


Saved: /content/drive/MyDrive/ECES650/kraken_out/evol1.kraken.report
Saved: /content/drive/MyDrive/ECES650/kraken_out/evol1.kraken.output


In [ ]:
# @title Output Parsing
# Parse Kraken2 report, summarize, export CSVs, and create a bar chart.
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display # For displaying dataframes richly in Colab

report_path = Path("/content/MyDrive/ECES650/evol1.kraken.report")
output_path = Path("/content/MyDrive/ECES650/evol1.kraken.output")

# Verify files exist
assert report_path.exists(), f"Missing report file: {report_path}"
assert output_path.exists(), f"Missing per-read output file: {output_path}"

# Kraken2 report columns
cols = ["percent","reads_clade","reads_taxon","rank","ncbi_taxid","name"]

# Read report (tab-separated)
df = pd.read_csv(report_path, sep="\t", header=None, names=cols, engine="python")

# Ensure types
df["percent"] = pd.to_numeric(df["percent"], errors="coerce")
df["reads_clade"] = pd.to_numeric(df["reads_clade"], errors="coerce")
df["reads_taxon"] = pd.to_numeric(df["reads_taxon"], errors="coerce")

# Clean indentation in names
df["name"] = df["name"].astype(str)
df["name"] = df["name"].str.strip()

# Compute classification stats
unclassified_mask = df["rank"] == "U"
unclassified_percent = float(df.loc[unclassified_mask, "percent"].sum()) if unclassified_mask.any() else 0.0
classified_percent = round(100.0 - unclassified_percent, 3)
unclassified_reads = int(df.loc[unclassified_mask, "reads_clade"].sum()) if unclassified_mask.any() else 0

# Estimate total reads if possible
total_reads_est = None
if unclassified_mask.any() and unclassified_percent > 0:
    total_reads_est = int(round(unclassified_reads / (unclassified_percent / 100.0)))
else:
    total_reads_est = int(df["reads_clade"].max()) if not df.empty else 0

# Species-level summary
species_df = df[df["rank"] == "S"].copy().sort_values("percent", ascending=False)
top_species = species_df.head(20).copy()

# Genus-level summary
genus_df = df[df["rank"] == "G"].copy().sort_values("percent", ascending=False)
top_genus = genus_df.head(15).copy()

# Save CSVs
top_species_csv = Path("/content/MyDrive/ECES650/evol1_top20_species.csv")
top_genus_csv = Path("/content/MyDrive/ECES650/evol1_top15_genus.csv")
summary_csv = Path("/content/MyDrive/ECES650/evol1_summary_counts.csv")

top_species.to_csv(top_species_csv, index=False)
top_genus.to_csv(top_genus_csv, index=False)

summary_df = pd.DataFrame([{
    "total_reads_estimate": total_reads_est,
    "unclassified_reads": unclassified_reads,
    "unclassified_percent": round(unclassified_percent, 3),
    "classified_percent": classified_percent
}])
summary_df.to_csv(summary_csv, index=False)

# Display interactive tables using standard Colab display
print("Kraken2 Summary (reads & percentages):")
display(summary_df)
print("\nTop 20 species by percent (clade):")
display(top_species[["name","percent","reads_clade","reads_taxon","ncbi_taxid"]])
print("\nTop 15 genera by percent (clade):")
display(top_genus[["name","percent","reads_clade","reads_taxon","ncbi_taxid"]])

# Plot bar chart (no custom colors)
plot_path = Path("/content/MyDrive/ECES650/evol1_top20_species.png")
plt.figure(figsize=(12,6))
plt.bar(top_species["name"], top_species["percent"])
plt.xticks(rotation=75, ha="right")
plt.ylabel("Percent of reads (clade)")
plt.title("Top 20 Species by Abundance (Kraken2)")
plt.tight_layout()
plt.savefig(plot_path)
plt.close()

# Provide file paths
print("Summary CSV:", summary_csv)
print("Top species CSV:", top_species_csv)
print("Top genus CSV:", top_genus_csv)
print("Top species bar chart:", plot_path)

AssertionError: Missing report file: /content/MyDrive/ECES650/evol1.kraken.report